In [ ]:
# 文档向量化测试
# count vectorizer

In [ ]:
a = "Many existing knowledge bases(KBs), including Freebase, Yago, and NELL, rely on a ﬁxed ontology, given as an input to the system, which deﬁnes the data to be cataloged in the KB, i.e., a hierarchy of categories and relations between them. The system then extracts facts that match the predeﬁned ontology. We propose an unsupervised model that jointly learns a latent ontological structure of an input corpus, and identiﬁes facts from the corpus that match the learned structure. Our approach combines mixed membership stochastic block models and topic models to infer a structure by jointly modeling text, a latent concept hierarchy, and latent semantic relationships among the entities mentioned in the text. As a case study, we apply the model to a corpus of Web documents from the software domain,and evaluate the accuracy of the various components of the learned ontology. "
b = ["hello world. what the fuck."]

In [ ]:
c = [" ".join(preprocess(a))]
vector = CountVectorizer(ngram_range=(1, 2), vocabulary=["knowledge base"], stop_words='english')
vector.build_analyzer()
x = vector.fit_transform(c)

In [ ]:
estimator = CountVectorizer(ngram_range=(1, 2), vocabulary=['fuck', 'hello', 'world', "hello world"], stop_words='english')
estimator.build_analyzer()
res = estimator.fit_transform(b)

In [ ]:
estimator.get_feature_names()

In [ ]:
res.toarray()
# estimator.vocabulary_

In [ ]:
# 斯坦福nlp工具测试
# stanfordcorenlp

In [42]:
from stanfordcorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost', port=9000)
sentence = 'Guangdong University of Foreign Studies is located in Guangzhou.'
print('Tokenize:', nlp.word_tokenize(sentence))
print('Part of Speech:', nlp.pos_tag(sentence))
print('Named Entities:', nlp.ner(sentence))
print('Constituency Parsing:', nlp.parse(sentence))
print('Dependency Parsing:', nlp.dependency_parse(sentence))

#nlp.close() # Do not forget to close! The backend server will consume a lot memery.

Tokenize: ['Guangdong', 'University', 'of', 'Foreign', 'Studies', 'is', 'located', 'in', 'Guangzhou', '.']
Part of Speech: [('Guangdong', 'NNP'), ('University', 'NNP'), ('of', 'IN'), ('Foreign', 'NNP'), ('Studies', 'NNPS'), ('is', 'VBZ'), ('located', 'JJ'), ('in', 'IN'), ('Guangzhou', 'NNP'), ('.', '.')]
Named Entities: [('Guangdong', 'ORGANIZATION'), ('University', 'ORGANIZATION'), ('of', 'ORGANIZATION'), ('Foreign', 'ORGANIZATION'), ('Studies', 'ORGANIZATION'), ('is', 'O'), ('located', 'O'), ('in', 'O'), ('Guangzhou', 'CITY'), ('.', 'O')]
Constituency Parsing: (ROOT
  (S
    (NP
      (NP (NNP Guangdong) (NNP University))
      (PP (IN of)
        (NP (NNP Foreign) (NNPS Studies))))
    (VP (VBZ is)
      (ADJP (JJ located)
        (PP (IN in)
          (NP (NNP Guangzhou)))))
    (. .)))
Dependency Parsing: [('ROOT', 0, 7), ('compound', 2, 1), ('nsubjpass', 7, 2), ('case', 5, 3), ('compound', 5, 4), ('nmod', 2, 5), ('auxpass', 7, 6), ('case', 9, 8), ('nmod', 7, 9), ('punct', 7, 10)]

In [ ]:
# 新闻数据集测试
# 20newsgroups

In [20]:
import newsgroups
size = 500
tmp = newsgroups.get_news_data(size)
newsdata = []
for cate in tmp:
    newsdata.extend(cate)

In [6]:
import preprocess as pp
import configs
import persister
texts = [' '.join(pp.preprocess_abstract(a)) for a in tmp]
persister.save_json(configs.NEWSDATA, texts)

In [23]:
import persister
import configs
texts = persister.load_json(configs.NEWSDATA)
newssent = persister.load_json(configs.NEWSSENT)

In [9]:
texts[0]

'regard fractal commpression seen fractal compressed movie fairly impressive first one wa gray scale movie casablanca wa mb minute fps video wa little grainy bad second one saw wa minute bit color fps measured mb consider fractal movie practical thing explore unlike many format end losing resolution know kind software hardware wa used creating movie saw guy showed said took minute per frame generate said playback wa frame per second else could put minute one floppy disk'

In [32]:
# 英文分句
# sent_tokenize

In [22]:
from nltk import sent_tokenize
newssent = []
for news in newsdata:
    newssent.append(pp.split2sent(news))
persister.save_json(configs.NEWSSENT, newssent)

In [41]:
newssent[160][4]

'Price is about $28, call to or visit in offices   in Menlo Park, in Reston, Virginia (800-USA-MAPS).'

In [36]:
print(nlp.dependency_parse(newssent[160][0]))

[('ROOT', 0, 1), ('punct', 1, 2), ('nmod:npmod', 4, 3), ('dep', 1, 4), ('punct', 4, 5), ('nummod', 10, 6), ('compound', 10, 7), ('compound', 10, 8), ('compound', 10, 9), ('dep', 4, 10), ('punct', 10, 11), ('compound', 13, 12), ('dep', 10, 13), ('punct', 15, 14), ('dep', 13, 15), ('nummod', 15, 16), ('punct', 15, 17), ('nummod', 15, 18), ('amod', 20, 19), ('dep', 15, 20), ('punct', 15, 21), ('nummod', 23, 22), ('dep', 15, 23), ('compound', 25, 24), ('nummod', 23, 25), ('punct', 15, 26)]


In [50]:
nlp.parse(newssent[0][0])

"(ROOT\r\n  (S\r\n    (SBAR (IN In)\r\n      (S\r\n        (VP (VBZ regards)\r\n          (PP (TO to)\r\n            (NP (JJ fractal) (NN commpression))))))\r\n    (, ,)\r\n    (NP (PRP I))\r\n    (VP (VBP have)\r\n      (VP (VBN seen)\r\n        (S\r\n          (NP (CD 2))\r\n          (ADJP (JJ fractal))\r\n          (S\r\n            (VP (VBN compressed)\r\n              (S (`` ``)\r\n                (NP (NNS movies))\r\n                ('' '')))))))\r\n    (. .)))"

In [44]:
pos = nlp.pos_tag(newssent[160][4])

In [55]:
s = "fuck\nm  e"
re.sub("\s+", " ", s)

'fuck m e'